<a href="https://colab.research.google.com/github/rodr1ggoql17/Procesamiento-Lenguaje-Natural/blob/main/Machine_Learning_y_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Detección de spam
Proceso mediante el cual se identifican y se filtran correos no deseados
* Importacia:
  * Seguridad y eficiecia
  * Experiencia de usuario

  

#Reglas de Naive Bayes
Es una fórmula que describe como actualizar las probabilidades de hipótesis cuando se dispone de nueva evidencia.

#Librerias

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install WordCloud

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from wordcloud import WordCloud

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CURSO NLP/data/spam.csv', encoding='ISO-8859-1')

In [ ]:
df

In [ ]:
count_label = df.groupby("label").count()
count_label

In [ ]:
df['label'].hist()

In [ ]:
#binarizamos las etiquetas del conjunto de datos
df['binary_label'] = df['label'].map({'ham': 0, 'spam':1})
Y = df['binary_label'].to_numpy()

In [ ]:
df

In [ ]:
Y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['email'], Y, test_size=0.33)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
vectores = CountVectorizer(decode_error='ignore')
Xtrain = vectores.fit_transform(x_train)
Xtest = vectores.transform(x_test)

In [ ]:
Xtrain

In [ ]:
Xtest

Entrenamiento y rendimiento del modelo

In [ ]:
model = MultinomialNB()
model.fit(Xtrain, y_train)
print('train acc: ', model.score(Xtrain, y_train))
print('test acc: ', model.score(Xtest, y_test))

In [ ]:
p_train = model.predict(Xtrain)
p_test = model.predict(Xtest)

print('train F1', f1_score(y_train, p_train))
print('test F1', f1_score(y_test, p_test))

obs: Si f1 train < f1 test, existe overfitting (sobreajuste)

In [ ]:
matrix_conf = confusion_matrix(y_train, p_train)
matrix_conf

In [ ]:
matrix_conf = confusion_matrix(y_test, p_test)
matrix_conf

In [ ]:
def plot_matrix(matrix):
  classes = ['ham', 'spam']
  df_cm = pd.DataFrame(matrix, index=classes, columns = classes)
  ax = sns.heatmap(df_cm, annot=True, fmt='g')
  ax.set_xlabel('predicted')
  ax.set_ylabel('target')
plot_matrix(matrix_conf)

In [ ]:
def visualizar(label):
  words = ''
  for msg in df[df['label']==label]['email']:
    msg = msg.lower()
    words += msg + ' '
  wordcloud = WordCloud(width=600, height = 400).generate(words)
  plt.imshow(wordcloud)
  plt.axis('off')
  plt.show()


In [ ]:
visualizar('spam')

In [ ]:
X = vectores.transform(df['email'])
df['predictions'] = model.predict(X)
df

In [ ]:
falso_spam = df[(df['predictions'] == 1) & (df['binary_label'] == 0)]['email']
for msg in falso_spam:
    print(msg)

In [ ]:
falso_real = df[(df['predictions'] == 0) & (df['binary_label'] == 1)]['email']
for msg in falso_real:
    print(msg)